In [4]:

# List of x and y coordinates
pt = []

# Calculate Dist. of point from line ay+bx+c=0 form
def calc_dist(a,b,c,x,y):
    p = (a*y) + (b*x) + c
    q = (a*a)+(b*b)
    q = np.sqrt(q)
    return abs(p/q)


# Divide DataSet and calculate e1/e2
def calc_error_ratio(size,k,m,c,pt):
    brk = size*k
    error = 0
    for i in range(brk):
        offset = calc_dist(1,m,c,pt[i][0],pt[i][1])
        error = error + offset
    e1 = error/brk
    error = 0
    for i in range(brk+1, size):
        offset = calc_dist(1,m,c,pt[i][0],pt[i][1])
        error = error + offset
    e2 = error/(size-brk)
    return e1/e2

# Calculate Slope
def calc_slope(m,c,size,ratio,unit):
    d_prv = 1-calc_error_ratio(size,ratio,m,c)
    while(d>0.001):
        m=m+unit
        d=1-calc_error_ratio(size,ratio,m,c)
        if(d>d_prv):
            # Terminate Looping and return previous value of m, Found best value of slope
            m_act = m - unit
            break;
        else:
            # Find other instance of m for a better suit
            d_prv = d
    
    return m_act,d

# Calculate Intercept
def calc_intercept(m,c,size,ratio,unit):
    d_prv = 1-calc_error_ratio(size,ratio,m,c)
    while(d>0.001):
        c=c+unit
        d=1-calc_error_ratio(size,ratio,m,c)
        if(d>d_prv):
            # Terminate Looping and return previous value of c, Found best value of intercept
            c_act = c - unit
            break;
        else:
            # Find other instance of c for a better suit
            d_prv = d
    
    return c_act,d


